In [4]:
import pandas as pd
df=pd.read_csv('imdb_dataset.csv')


In [5]:
df

,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0
...,...,...
24995,A hit at the time but now better categorised a...,1
24996,I love this movie like no other. Another time ...,1
24997,This film and it's sequel Barry Mckenzie holds...,1
24998,'The Adventures Of Barry McKenzie' started lif...,1


In [6]:
import pandas as pd

# Supposons que df est votre DataFrame

# Vérifier s'il y a des valeurs NaN dans le DataFrame
has_nan = df.isna().values.any()

if has_nan:
    print("Le DataFrame contient des valeurs NaN.")
else:
    print("Le DataFrame ne contient pas de valeurs NaN.")

Le DataFrame ne contient pas de valeurs NaN.


In [7]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

nltk.download('punkt')
nltk.download('stopwords')

def normalize_phrase(df, column_name):
    # Tokenisation
    df['Tokenized_Phrase'] = df['text'].apply(lambda x: word_tokenize(x.lower(), language='english'))

    # Suppression des mots vides (stopwords)
    stop_words = set(stopwords.words('english'))
    df['Tokenized_Phrase'] = df['Tokenized_Phrase'].apply(lambda x: [word for word in x if word not in stop_words])

    # Normalisation des mots (stemming)
    stemmer = SnowballStemmer('english')
    df['Tokenized_Phrase'] = df['Tokenized_Phrase'].apply(lambda x: [stemmer.stem(word) for word in x])

    return df
df = normalize_phrase(df, 'text')
df

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\abdou\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abdou\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text,label,Tokenized_Phrase
0,I rented I AM CURIOUS-YELLOW from my video sto...,0,"[rent, curious-yellow, video, store, controver..."
1,"""I Am Curious: Yellow"" is a risible and preten...",0,"[``, curious, :, yellow, '', risibl, pretenti,..."
2,If only to avoid making this type of film in t...,0,"[avoid, make, type, film, futur, ., film, inte..."
3,This film was probably inspired by Godard's Ma...,0,"[film, probabl, inspir, godard, 's, masculin, ..."
4,"Oh, brother...after hearing about this ridicul...",0,"[oh, ,, brother, ..., hear, ridicul, film, ump..."
...,...,...,...
24995,A hit at the time but now better categorised a...,1,"[hit, time, better, categoris, australian, cul..."
24996,I love this movie like no other. Another time ...,1,"[love, movi, like, ., anoth, time, tri, explai..."
24997,This film and it's sequel Barry Mckenzie holds...,1,"[film, 's, sequel, barri, mckenzi, hold, ,, tw..."
24998,'The Adventures Of Barry McKenzie' started lif...,1,"[the, adventur, barri, mckenzi, ', start, life..."


In [9]:
X=df['Tokenized_Phrase']
Y=df['label']

In [12]:
Y.shape

(25000,)

In [13]:
from sklearn.model_selection import train_test_split
seed=1234
x_train,x_test, y_train ,y_test =train_test_split(X,Y,test_size=0.2, random_state=seed)

In [15]:
x_test.shape

(5000,)

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

# Conversion de chaque élément de x_train en chaîne de caractères
x_train_str = [str(text) for text in x_train]

# Vectorisation des données d'entraînement
vectorizer = CountVectorizer()
x_train_vec = vectorizer.fit_transform(x_train_str)

# Vectorisation des données de test en utilisant le même vocabulaire que les données d'entraînement
x_test_str = [str(text) for text in x_test]
x_test_vec = vectorizer.transform(x_test_str)


In [19]:

# Entraînement du modèle RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(x_train_vec, y_train)

# Calcul du pourcentage de précision


RandomForestClassifier()

In [21]:
accuracy = rf.score(x_test_vec, y_test)
accuracy

0.839

In [22]:
from sklearn.metrics import classification_report
print(classification_report(y_test,rf .predict(x_test_vec)))

              precision    recall  f1-score   support

           0       0.84      0.84      0.84      2488
           1       0.84      0.84      0.84      2512

    accuracy                           0.84      5000
   macro avg       0.84      0.84      0.84      5000
weighted avg       0.84      0.84      0.84      5000



In [23]:
from sklearn.model_selection import cross_val_score
score=cross_val_score(rf,x_train_vec,y_train, cv=3)
score

array([0.84475776, 0.84835758, 0.83948395])

In [24]:
rf.predict(vectorizer.transform([ 'A gripping and intense thriller that keeps you on the edge of your seat from start to finish. The plot twists and turns are masterfully executed, and the performances are top-notch. A must-watch for fans of suspenseful cinema.']))

array([1], dtype=int64)

In [25]:
rf.predict(vectorizer.transform(['A heartwarming and beautifully crafted coming-of-age story that captures the essence of friendship and self-discovery. The cinematography is stunning, and the young cast delivers outstanding performances. This film will leave you with a smile on your face and a sense of nostalgia']))

array([1], dtype=int64)

In [26]:
rf.predict(vectorizer.transform(['Unfortunately, this film was a big disappointment. The plot was confusing and predictable, with unconvincing twists. The performances of the actors felt forced and lacked depth. Furthermore, the direction was mediocre, with questionable choices in filmmaking. Overall, it was a disappointing experience that didn t live up to expectations']))

array([0], dtype=int64)

In [27]:
rf.predict(vectorizer.transform(['This film had the potential to be interesting, but it fell flat. The story was poorly developed, with underdeveloped characters and dull dialogue. The special effects were unconvincing and seemed artificial. Additionally, the pacing of the film was slow and lacked suspense. It was a complete letdown, and I wouldnt recommend it']))

array([0], dtype=int64)

In [31]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
# Initialiser le modèle Naive Bayes
model = MultinomialNB()

# Entraîner le modèle
model.fit(x_train_vec,y_train)

# Faire des prédictions sur l'ensemble de test
y_pred = model.predict(x_test_vec)

# Évaluer les performances du modèle
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8522


In [32]:
model.predict(vectorizer.transform([ 'A gripping and intense thriller that keeps you on the edge of your seat from start to finish. The plot twists and turns are masterfully executed, and the performances are top-notch. A must-watch for fans of suspenseful cinema.']))

array([1], dtype=int64)

In [33]:
model.predict(vectorizer.transform(['Unfortunately, this film was a big disappointment. The plot was confusing and predictable, with unconvincing twists. The performances of the actors felt forced and lacked depth. Furthermore, the direction was mediocre, with questionable choices in filmmaking. Overall, it was a disappointing experience that didn t live up to expectations']))

array([0], dtype=int64)

In [35]:

from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()

# Entraînement du modèle
logreg.fit(x_train_vec,y_train)

# Prédictions sur les données de test
y_pred = logreg.predict(x_test_vec)

# Évaluation du modèle


c:\Users\abdou\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [36]:
accuracy = accuracy_score(y_test, y_pred)
print("Exactitude (Accuracy) : ", accuracy)


Exactitude (Accuracy) :  0.8666
